# The Kalman Filter (KF)

The KF is one of the most widely used algorithms in time-series applications, such as tracking objects or state estimation in robot navigation. In this notebook, we'll provide a brief overview of the main theory underlying the KF and then demonstrate its utility in an application where we track an object moving in $\mathbb{R}^3$ space, given noisy measurements of its location.

Below are the necessary imports for this notebook:

In [1]:
# Widget to manipulate plots in Jupyter notebooks
%matplotlib widget 

import matplotlib.pyplot as plt # For general plotting

# GPU suitable form of NumPy, also works on the CPU
from jax import numpy as jnp
from jax import random as jr
from jax import vmap

# JAX library for SSMs
from dynamax.utils.plotting import plot_uncertainty_ellipses
from dynamax.linear_gaussian_ssm import LinearGaussianSSM
from dynamax.linear_gaussian_ssm import lgssm_smoother, lgssm_filter

jnp.set_printoptions(suppress=True)

plt.rc('font', size=22)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=22)   # fontsize of the figure title

2023-05-21 19:32:48.386158: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


<b>Aside:</b> please note the inclusion of [JAX](https://github.com/google/jax) and [Dynamax](https://github.com/probml/dynamax). JAX is akin to NumPy in syntax and use, but processes instructions on the GPU, efficiently computes gradients (autograd), and reaps the benefits of [just-in-time (JIT) compilation](https://en.wikipedia.org/wiki/Just-in-time_compilation) (e.g., [Numba](https://numba.pydata.org/)), which are all advantageous features for machine learning coding. 

Dynamax is a library written in JAX for state space models (SSMs). The KF is an algorithm that applies to a special type of SSM: a Linear-Gaussian SSM (LG-SSM). 

You should follow the [installation instructions](https://github.com/google/jax#installation) for JAX (feel free to use CPU/GPU in this notebook). You can also refer [here](https://github.com/cloudhan/jax-windows-builder) if you're a Windows user.

## Posterior Inference in SSMs

Consider the problem of tracking the internal state vector of a dynamical system in the presence of noise and other inaccuracies using:
- Knowledge of the system equations; and
- Measured outputs or observations in time.

A popular model of sequential data suitable for this type of problem is a <b>state space model (SSM)</b>. To capture the "internal state" (or hidden state) of a system, we often think of latent variable models, e.g., [hidden Markov models (HMMs)](https://en.wikipedia.org/wiki/Hidden_Markov_model), and a means of deriving these latent variables from observable data, such as [PCA](../unsupervised_learning/pca_dim_reduction.ipynb). For a dynamical system in particular, it is appropriate to adopt latent variable <i>sequence</i> models, where HMMs and SSMs are typical examples of this class of chain-structured graphical models. We thus opt to utilize an SSM for our tracking problem.

More specifically, an SSM is a <b>partially observed Markov model</b>, where the hidden state evolves over time according to a Markov process (assumption that future states of the process depend only upon the present state), and these hidden states generate observations at each timestep. An SSM can be represented as a stochastic discrete-time dynamical system of the form:
\begin{align*}
\mathbf{z}_t &= \mathbf{f}(\mathbf{z}_{t-1}, \mathbf{u}_t, \mathbf{q}_t) \\
\mathbf{x}_t &= \mathbf{h}(\mathbf{z}_{t}, \mathbf{u}_t, \mathbf{r}_t) 
\end{align*}
where $\mathbf{z}_t \in \mathbb{R}^{N_z}$ is the true or hidden state at time $t$, $\mathbf{x}_t \in \mathbb{R}^{N_x}$ is the observation (output), and $\mathbf{u}_t \in \mathbb{R}^{N_u}$ is an optional control input. The function $\mathbf{f}$ is the state <b>transition</b> or <b>dynamics function</b>, and $\mathbf{h}$ is the <b>observation function</b>, both of which are possibly non-linear functions. Lastly, $\mathbf{q}_t$ is the <b>process noise</b> and $\mathbf{r}_t$ is the <b>observation noise</b>.

In an SSM, two common time-series tasks are 1) to infer the hidden states of the model given incoming observations; and 2) use the model to predict future observations. Our main objective here is the former task, where we wish to find the true internal state of the dynamical system given a sequence of observations. Rather than expressing the SSM as a deterministic function of random noise, we can also rewrite its equations in a probabilistic form:
\begin{align*}
p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1}, \mathbf{u}_t) &= p(\mathbf{z}_t \, | \, \mathbf{f}(\mathbf{z}_{t-1}, \mathbf{u}_t))  \\
p(\mathbf{x}_t \, | \, \mathbf{z}_t, \mathbf{u}_t) &= p(\mathbf{x}_t \, | \, \mathbf{h}(\mathbf{z}_{t}, \mathbf{u}_t))
\end{align*}
where $p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1}, \mathbf{u}_t)$ is the <b>transition model</b> and $p(\mathbf{x}_t \, | \, \mathbf{z}_t, \mathbf{u}_t)$ is the <b>observation model</b>. 

Unrolling over time, we can write the corresponding joint distribution of the SSM as:
\begin{equation*}
p(\mathbf{x}_{1:T}, \mathbf{z}_{1:T} \, | \, \mathbf{u}_{1:T}) = \left[ p(\mathbf{z}_1 \, | \, \mathbf{u}_1) \prod_{t=2}^T p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1}, \mathbf{u}_t) \right] \left[ \prod_{t=1}^T p(\mathbf{x}_t \, | \, \mathbf{z}_{t}, \mathbf{u}_t) \right],
\end{equation*}
where $p(\mathbf{z}_1 \, | \, \mathbf{u}_1)$ is the prior or initial state distribution, sometimes denoted as $p(\mathbf{z}_0)$. When there are no external inputs, as we will assume for the remainder of this notebook, then the joint distribution further simplifies to:
\begin{equation*}
p(\mathbf{x}_{1:T}, \mathbf{z}_{1:T}) = \left[ p(\mathbf{z}_1) \prod_{t=2}^T p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1}) \right] \left[ \prod_{t=1}^T p(\mathbf{x}_t \, | \, \mathbf{z}_{t}) \right].
\end{equation*}
The SSM joint distribution can be written in the above form due to two assumptions. First, the Markov assumption that the true state depends only on the immediately previous state, i.e., $p(\mathbf{z}_t \, | \, \mathbf{z}_{1}, \ldots,\mathbf{z}_{t-1}) = p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1})$. Second, the assumption that the current measurement, $\mathbf{x}_{t}$, is conditionally independent of all past observations, $\mathbf{x}_{1:t-1}$, given the current hidden state, $\mathbf{z}_{t}$.

Viewing the SSM described above from a Bayesian perspective, our task of finding the true state of the system has now been reframed as performing posterior inference about the hidden states, also known as <b>state estimation</b>. For example, consider an object moving around in 2D space. Our aim is to estimate its location and velocity $\mathbf{z}_t \in \mathbb{R}^4$ given noisy sensor measurements of its location $\mathbf{x}_t \in \mathbb{R}^2$. A <b>linear-Gaussian SSM (LG-SSM)</b> is a suitable model for this task, where we must compute the <b>belief state</b>, $p(\mathbf{z}_t \, | \, \mathbf{x}_{1:t})$, i.e., our posterior of interest. This is known as <b>Bayesian filtering</b>. If the belief state is represented as a Gaussian, then the KF is capable of solving this task, as we will see in the [later tracking application](#Tracking-an-Object-using-a-KF). While filtering is performed <i>online</i>, meaning inference is performed using only the data observed up to $t$, another more accurate state estimation technique for <i>offline</i> use is known as <b>Bayesian smoothing</b>. In this setting, the aim is to compute $p(\mathbf{z}_t \, | \, \mathbf{x}_{1:T})$, where all the data in the sequence up to time $T$ has already been observed. 

Please refer to Chapter 8 of Murphy's "Probabilistic Machine Learning: Advanced Topics" book for more background information on Gaussian filtering and smoothing, as we will only glance at filtering and the KF in this notebook.

## Bayesian Filtering

[Recursive Bayesian estimation](https://en.wikipedia.org/wiki/Recursive_Bayesian_estimation), also known as the <b>Bayes filter</b>, is an algorithm to estimate an unknown probability density function recursively over time using incoming measurements and a known model. In other words, we can use a Bayes filter to recursively compute the belief state, $p(\mathbf{z}_t \, | \, \mathbf{x}_{1:t})$, given the prior belief from the previous step, $p(\mathbf{z}_{t-1} \, | \, \mathbf{x}_{1:t-1})$, the new observation, $\mathbf{x}_{t}$, and our known mathematical process model. Sequential Bayesian updating can be performed at constant time complexity to recursively estimate this belief state. For a dynamical system, the overall procedure boils down to a <b>predict-update</b> cycle.

In the <b>prediction step</b>, we have the <b>Chapman-Kolmogorov equation</b> to compute the one-step-ahead predictive distribution for the hidden state:
\begin{equation*}
p(\mathbf{z}_{t} \, | \, \mathbf{x}_{1:t-1}) = \int p(\mathbf{z}_{t} \, | \, \mathbf{z}_{t-1})p(\mathbf{z}_{t-1} \, | \, \mathbf{x}_{1:t-1})d\mathbf{z}_{t-1}.
\end{equation*}
This equation updates the posterior from the previous time step, $p(\mathbf{z}_{t-1} \, | \, \mathbf{x}_{1:t-1})$, into the prior for the current step, $p(\mathbf{z}_{t} \, | \, \mathbf{x}_{1:t-1})$.

The <b>update step</b> is then simply Bayes' rule:
\begin{equation*}
p(\mathbf{z}_{t} \, | \, \mathbf{x}_{1:t}) = \frac{1}{Z_t} p(\mathbf{x}_{t} \, | \, \mathbf{z}_{t})p(\mathbf{z}_{t} \, | \, \mathbf{x}_{1:t-1}),
\end{equation*}
where the normalization constant, $Z_t$, is:
\begin{equation*}
Z_t = \int p(\mathbf{x}_{t} \, | \, \mathbf{z}_{t})p(\mathbf{z}_{t} \, | \, \mathbf{x}_{1:t-1})d\mathbf{z}_{t} = p(\mathbf{x}_{t} \, | \, \mathbf{x}_{1:t-1}).
\end{equation*}

The normalization constants can also be utilized to provide the log-likelihood of the sequence:
\begin{equation*}
\log p(\mathbf{x}_{1:T}) = \sum_{t=1}^T \log p(\mathbf{x}_{t} \, | \, \mathbf{x}_{1:t-1}) = \sum_{t=1}^T \log Z_t,
\end{equation*}
where we define $p(\mathbf{x}_{1} \, | \, \mathbf{x}_{0}) = p(\mathbf{x}_{1})$. This quantity is the sum of log probabilities of the one-step-ahead measurement predictions, so it can be regarded as a measure of how "surprised" the model is at each timestep.

Next let's examine the KF, an algorithm for <i>exact</i> Bayesian filtering in a specific type of SSM: linear-Gaussian SSMs (LG-SSMs). 

## KF Formulation

Computing the integrals required to implement Bayesian filtering is generally intractable. However, if the state space is <i>discrete</i>, as in an HMM, or if the SSM is a linear-Gaussian model, then the belief state can be represented by a distribution with a closed-form solution. The KF is an algorithm that specifically operates on the latter case of the LG-SSM, where all the distributions are linear-Gaussian.

For the KF to apply, we will also assume that the SSM used to represent our dynamical system can be described by the following equations:
\begin{align*}
\mathbf{z}_t &= \mathbf{f}(\mathbf{z}_{t-1}, \mathbf{u}_t) + \mathbf{q}_t \\
\mathbf{x}_t &= \mathbf{h}(\mathbf{z}_{t}, \mathbf{u}_t) + \mathbf{r}_t 
\end{align*}
where $\mathbf{f}$ is a known <b>linear</b> transition function, $\mathbf{h}$ is a known <b>linear</b> observation function, $\mathbf{q}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{Q}_t)$ is additive process noise assumed to be drawn from a zero-mean multivariate Gaussian with known covariance $\mathbf{Q}_t$, and similarly $\mathbf{r}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{R}_t)$ is observation noise with a known covariance matrix $\mathbf{R}_t$. For simplicity in this example, we will also assume no external inputs $\mathbf{u}_t$ going forward.

The above LG-SSM, also known as a <b>linear dynamical system (LDS)</b>, is written in its deterministic form as:
\begin{align*}
\mathbf{z}_t &= \mathbf{F}_t\mathbf{z}_{t-1} + \mathbf{q}_t \\
\mathbf{x}_t &= \mathbf{H}_t\mathbf{z}_{t} + \mathbf{r}_t 
\end{align*}
where $\mathbf{F}_t$ and $\mathbf{H}_t$ are known state transition and observation matrices, respectively.

Or alternatively, the LG-SSM can be written in its probabilistic form with all the probability densities represented as Gaussian:
\begin{align*}
p(\mathbf{z}_t \, | \, \mathbf{z}_{t-1}) &= \mathcal{N}(\mathbf{z}_t \, | \, \mathbf{F}_t\mathbf{z}_{t-1}, \mathbf{Q}_t)  \\
p(\mathbf{x}_t \, | \, \mathbf{z}_t) &= \mathcal{N}(\mathbf{x}_t \, | \, \mathbf{H}_t\mathbf{z}_{t}, \mathbf{R}_t)
\end{align*}

The subsequent sections describe the predict-update cycle for the KF under this model, where the belief state at time $t$ is now given as $p(\mathbf{z}_t \, | \, \mathbf{x}_{1:t}) = \mathcal{N}(\mathbf{z}_t \, | \, \boldsymbol{\mu}_{t|t}, \boldsymbol{\Sigma}_{t|t})$, with $\boldsymbol{\mu}_{t|t}$ and $\boldsymbol{\Sigma}_{t|t}$ denoting the posterior mean and covariance given observations $\mathbf{x}_{1:t}$. You may also find $\boldsymbol{\mu}_{t|t}$ written as $\boldsymbol{m}_{t|t}$ and $\boldsymbol{\Sigma}_{t|t}$ written as $\boldsymbol{P}_{t|t}$ in other sources. Please note that the following steps will contain no derivation of their equations (see Chapter 8 of Murphy's advanced topics book for the KF derivation).

### Predict Step

The one-step-ahead prediction step of the hidden state, or the <b>time update step</b>, is equated as follows:
\begin{align*}
p(\mathbf{z}_t \, | \, \mathbf{x}_{1:t-1}) & = \mathcal{N}(\mathbf{z}_t \, | \, \boldsymbol{\mu}_{t|t-1}, \boldsymbol{\Sigma}_{t|t-1}) \\
\boldsymbol{\mu}_{t|t-1} &= \mathbf{F}_t\boldsymbol{\mu}_{t-1|t-1} \\
\boldsymbol{\Sigma}_{t|t-1} &= \mathbf{F}_t\boldsymbol{\Sigma}_{t-1|t-1}\mathbf{F}_t^\intercal + \mathbf{Q}_t
\end{align*}
where $\boldsymbol{\mu}_{t|t-1}$ and $\boldsymbol{\Sigma}_{t|t-1}$ are the predicted <i>a priori</i> state mean and covariance estimates.

### Update Step

The update step, or the <b>measurement step</b>, is then computed using Bayes' rule:
\begin{align*}
p(\mathbf{z}_t \, | \, \mathbf{x}_{1:t}) & = \mathcal{N}(\mathbf{z}_t \, | \, \boldsymbol{\mu}_{t|t}, \boldsymbol{\Sigma}_{t|t}) \\
\mathbf{S}_{t} &= \mathbf{H}_t\boldsymbol{\Sigma}_{t|t-1}\mathbf{H}_t^\intercal + \mathbf{R}_t \\
\mathbf{K}_{t} &= \boldsymbol{\Sigma}_{t|t-1}\mathbf{H}_t^\intercal\mathbf{S}_t^{-1} \\
\boldsymbol{\mu}_{t|t} &= \boldsymbol{\mu}_{t|t-1} + \mathbf{K}_t(\mathbf{x}_t - \mathbf{H}_t\boldsymbol{\mu}_{t|t-1})  \\
\boldsymbol{\Sigma}_{t|t} &= \boldsymbol{\Sigma}_{t|t-1} - \mathbf{K}_{t}\mathbf{H}_{t}\boldsymbol{\Sigma}_{t|t-1}
\end{align*}
where $\mathbf{K}_{t}$ is the <b>Kalman gain matrix</b>, while $\boldsymbol{\mu}_{t|t}$ and $\boldsymbol{\Sigma}_{t|t}$ are the updated <i>a posteriori</i> state mean and covariance estimates. The difference term $\mathbf{e}_t = \mathbf{x}_t - \mathbf{H}_t\boldsymbol{\mu}_{t|t-1}$ is also referred to as the <b>residual error</b> or the <b>innovation</b>.

## Tracking an Object using a KF

## Variants of the KF


There are also other inference methods for SSMs.